Importing all necessary libraries here:

In [129]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


Reading the dataset i.e. training data, there are two files:
-   train_dataset.csv which only contains employee data that the model can be trained upon.
-   train_salaries.csv contains target variable data i.e. Salaries of employees.

In [130]:
X = pd.read_csv("./datasets/train_dataset.csv")
X = X[:100000]
X.head()

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73
2,JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38
3,JOB1362684407690,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17
4,JOB1362684407691,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16


In [131]:
y = pd.read_csv("./datasets/train_salaries.csv")
y=y[:100000]
print(y.head())
y = y.drop(columns=["jobId"])


              jobId  salary
0  JOB1362684407687     130
1  JOB1362684407688     101
2  JOB1362684407689     137
3  JOB1362684407690     142
4  JOB1362684407691     163


I shall try and check for null values, while understaning the data better.

In [132]:
print(X.columns)
print("\n")
print(X.describe())
print("\n")
print("The total number of Unique jobs are: "+str(len(X["jobId"].unique())))

Index(['jobId', 'companyId', 'jobType', 'degree', 'major', 'industry',
       'yearsExperience', 'milesFromMetropolis'],
      dtype='object')


       yearsExperience  milesFromMetropolis
count     100000.00000        100000.000000
mean          11.94206            49.373400
std            7.20478            28.876814
min            0.00000             0.000000
25%            6.00000            24.000000
50%           12.00000            49.000000
75%           18.00000            74.000000
max           24.00000            99.000000


The total number of Unique jobs are: 100000


I personally feel that jobId does not play a huge role if we are not predicting salary within a organization. What I mean to say, is the fact that jobId acts as a primary key within a data, and has no context in this regard where we already know the necessary features.


In [133]:
X = X.drop(columns=["jobId"])
X.columns

Index(['companyId', 'jobType', 'degree', 'major', 'industry',
       'yearsExperience', 'milesFromMetropolis'],
      dtype='object')

So, there are eight input features. And One target variable.

In [134]:
cat_data = X.select_dtypes(include=["object"])
cat_data.columns

Index(['companyId', 'jobType', 'degree', 'major', 'industry'], dtype='object')

Lets dropna values in both X and y and perform some encoding for X.

In [135]:
X= X.dropna()
X

,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,COMP37,CFO,MASTERS,MATH,HEALTH,10,83
1,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73
2,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38
3,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17
4,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16
...,...,...,...,...,...,...,...
99995,COMP60,CTO,NONE,NONE,FINANCE,8,1
99996,COMP47,JUNIOR,HIGH_SCHOOL,NONE,SERVICE,20,45
99997,COMP41,CEO,DOCTORAL,ENGINEERING,FINANCE,21,94
99998,COMP31,CTO,DOCTORAL,BUSINESS,SERVICE,12,74


In [136]:

y = y.dropna()
y

,salary
0,130
1,101
2,137
3,142
4,163
...,...
99995,138
99996,76
99997,194
99998,113


In [149]:
oneHencoder = OneHotEncoder()

X_encoded = oneHencoder.fit_transform(cat_data)
X_encoded = pd.DataFrame(X_encoded.toarray())
X_encoded.columns = X_encoded.columns.astype(str)
print("Dimensions encoded X frame are:"+str(X_encoded.shape[1]))
X_encoded.head()



Total col within encoded X frame are:92


,0,1,2,3,4,5,6,7,8,9,...,82,83,84,85,86,87,88,89,90,91
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Time to split this data!

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33)

ValueError: Found input variables with inconsistent numbers of samples: [200000, 100000]